# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [72]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Suha\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [139]:
 pip install newsapi-python

In [150]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='55135490a1184b1499db322a8c65d6be')

In [151]:
api_key = os.getenv("news_api")

In [152]:
newsapi = NewsApiClient(api_key="55135490a1184b1499db322a8c65d6be")

In [153]:
btc_headlines = newsapi.get_everything(
    q="bitcoin",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [154]:
# Fetch all the news about Ethereum
eth_headlines = newsapi.get_everything(
    q="ethereum",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [155]:
btc_sentiments = []

for article in btc_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
btc_df = pd.DataFrame(btc_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
btc_df = btc_df[cols]

btc_df.head()

,date,text,compound,positive,negative,neutral
0,2021-05-12,Just weeks after Tesla started accepting Bitco...,0.3818,0.071,0.00,0.929
1,2021-05-07,Shares of Square are up more than 6% today aft...,0.2960,0.058,0.00,0.942
2,2021-05-12,Image: Tesla\r\n\n \n\n Tesla has stopped acce...,0.4939,0.134,0.05,0.816
3,2021-05-19,Illustration by Alex Castro / The Verge\r\n\n ...,0.0000,0.000,0.00,1.000
4,2021-05-11,"Mark Zuckerberg posted a picture of his two, f...",0.8455,0.217,0.00,0.783


In [156]:
eth_sentiments = []

for article in eth_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
eth_df = pd.DataFrame(eth_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
eth_df = eth_df[cols]

eth_df.head()

,date,text,compound,positive,negative,neutral
0,2021-05-12,"Vitalik Buterin, the creator of Ethereum, on W...",0.2263,0.060,0.00,0.940
1,2021-05-04,Their investors call them disruptive innovator...,-0.4019,0.072,0.15,0.778
2,2021-05-15,Solana isn’t known yet outside of the crypto c...,0.4019,0.083,0.00,0.917
3,2021-05-01,New York (CNN Business)Bitcoin prices continue...,0.0000,0.000,0.00,1.000
4,2021-05-03,"The creators behind CryptoPunks, one of the mo...",0.4754,0.091,0.00,0.909


In [157]:
btc_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.00000,100.000000
mean,0.029610,0.049310,0.04154,0.909170
std,0.398529,0.065435,0.05215,0.078247
min,-0.762700,0.000000,0.00000,0.677000
25%,-0.273200,0.000000,0.00000,0.858750
50%,0.000000,0.000000,0.00000,0.918500
75%,0.323650,0.075750,0.07150,1.000000
max,0.845500,0.275000,0.20300,1.000000


In [158]:
eth_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.077482,0.053400,0.035040,0.901540
std,0.356690,0.059105,0.060397,0.121939
min,-0.918600,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.849250
50%,0.000000,0.049000,0.000000,0.923500
75%,0.340000,0.081250,0.065500,1.000000
max,0.822500,0.256000,0.289000,1.000000


---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [11]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re


In [ ]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [ ]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')

#Add additional stop words
stop.append("u")
stop.append("hey")

stop = set(stop)

In [7]:
def stopwords():
    pass

In [3]:
def WordNetLemmatizer():
    pass

In [ ]:
#Set tokenization function
def clean_text(article):
    
    words = word_tokenize(article)
    
    words = list(filter(lambda w:w.lower(), words))
 
    words = list(filter(lambda t:t not in punctuation, words))
    
    words = list(filter(lambda t: t.lower() not in stop, words))
    
    
    token = [lemmatizer.lemmatize(word) for word in words]
    
    return token

In [ ]:
btc_df['token'] = btc_df['text'].apply(clean_text)

In [ ]:
eth_df['token'] = eth_df['text'].apply(clean_text)

In [ ]:
eth_df.head()

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
#Generate Bitcoin N-grams where N=2
N = 2
grams = ngrams(clean_text(btc_df.text.str.cat()), N)
Counter(grams).most_common(10) #Use token_count to get the top 10 words for Bitcoin

In [ ]:
#Generate Ethereum N-grams where N=2
N = 2
gram1 = ngrams(clean_text(eth_df.text.str.cat()), N)
Counter(gram1).most_common(10) #Use token_count to get the top 10 words for ETH

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
#Generate the Bitcoin word cloud

df_cloud_btc = WordCloud().generate(btc_df.text.str.cat())
plt.imshow(df_cloud_btc)
plt.show()

In [ ]:
#Generate the Bitcoin word cloud

df_cloud_eth = WordCloud().generate(eth_df.text.str.cat())
plt.imshow(df_cloud_eth)
plt.show()

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
!python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
all_btc_text = btc_df.text.str.cat()

In [ ]:
# Run the NER processor on all of the text
doc = nlp(all_btc_text)

# Add a title to the document
doc.user_data['title'] = "Bitcoin NER"

In [ ]:
# Render the visualization
displacy.render(doc,style='ent',jupyter=True)

In [ ]:
#List all Entities

for ent in doc.ents:
    print(ent.text, ent.label_)

---

### Ethereum NER

In [ ]:
# Concatenate all of the ETH text together
all_eth_text = eth_df.text.str.cat()

In [ ]:
# Run the NER processor on all of the text
doc2 = nlp(all_eth_text)

# Add a title to the document
doc2.user_data['title'] = "Ethereum NER"

In [ ]:
# Render the visualization
displacy.render(doc2,style='ent',jupyter=True)

In [ ]:
#List all Entities

for ent in doc2.ents:
    print(ent.text, ent.label_)

---